In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from random import randint

# own Modules 
from models_mse import LstmMse
from data_set import DataSet
from cross_validation import CrossValidationProvider
from scaler import DataScaler
from trainer import Trainer
from loss_module import LossMse
from tester import Tester
from logger import Logger

## Paramerters artifical data

In [17]:
param = {
    "data" : {
        "path" : '../../../../data/artifical_signals/artifical_2_signals.csv' ,
    },
    "preprocessing" : {
        "droped_features": ["ID"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "input_size" : 2,
        "n_hidden_lstm" : [20], 
        "sequence_size" : [10,20], 
        "batch_size" : 8,
        "lstm_layer" : [1],
        "n_hidden_fc": [20,40],
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2
    },
    "cycling_lr" : {
        "scheduler_active" : True, 
        # Mode can be one of {triangular, triangular2, exp_range}
        "mode" : "triangular", 
        "gamma" : 0.9995,
        "base_lr" : 0.0001, # 0.016, 
        "max_lr" :0.0005,  # 0.75
    },
    "training": {
        "stake_training_data": 0.75,
        "total_number" : 6000,
        "n_folds_cv": 2,
        "n_epochs" : 2,
        "patience" : 7,
    },
    "filed_location": {
        "model_for_test" : "../../../../models/cross_validation/MSE/artifical_data_",
        "log_file" : "../../../../models/cross_validation/MSE/artifical_data_log_", 
        "history_trainval" : "../../../visualisation/files/cross_validation/MSE/artifical_data_trainval.csv",
        "history_best_configuration" : "../../../visualisation/files/cross_validation/MSE/artifical_data_configurations.csv"
    }
}

## Paramerters cpps data

In [5]:
param = {
    "data" : {
        "path" : '../../../../data/cpps_degradation_new/data_obs10/train/obs_space_train_sinusiod_preprocessed.csv',
    },
    "preprocessing" : {
        "droped_features": ["ID"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "input_size" : 10,
        "n_hidden_lstm" : [10,50], 
        "sequence_size" : [10,50], 
        "batch_size" : 8,
        "lstm_layer" : [1],
        "n_hidden_fc": [60],
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2
    },
    "cycling_lr" : {
        "scheduler_active" : True, 
        # Mode can be one of {triangular, triangular2, exp_range}
        "mode" : "triangular", 
        "gamma" : 0.9995,
        "base_lr" : 0.0001, # 0.016, 
        "max_lr" :0.0005,  # 0.75
    },
    "training": {
        "stake_training_data": 0.75,
        "total_number" : 6000,
        "n_folds_cv": 2,
        "n_epochs" : 4,
        "patience" : 7,
    },
    "filed_location": {
        "model_for_test" : "../../../../models/cross_validation/MSE/cpps_",
        "log_file" : "../../../../models/cross_validation/MSE/cpps_log_", 
        "history_trainval" : "../../../visualisation/files/cross_validation/MSE/cpps_trainval.csv",
        "history_best_configuration" : "../../../visualisation/files/cross_validation/MSE/cpps_configurations.csv"
    }
}

## Paramerters phm data

In [ ]:
param = {
    "data" : {
        "path" : '../../../../data/phm_data_challenge/recipe/dataset_for_each_recipe/training/training_recipe_67.csv' ,
    },
    "preprocessing" : {
        "droped_features": ["ID", "stage", "Lot", "runnum", "recipe", "recipe_step",
                            "up time", "ongoing time", 
                            "ETCHSOURCEUSAGE", "ETCHAUXSOURCETIMER", 
                            "ETCHAUX2SOURCETIMER", "FIXTURESHUTTERPOSITION", "ROTATIONSPEED"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "input_size" : 12,
        "n_hidden_lstm" : [10,20], 
        "sequence_size" : [10,30], 
        "batch_size" : 8,
        "lstm_layer" : [1],
        "n_hidden_fc": [20,60],
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2
    },
    "cycling_lr" : {
        "scheduler_active" : True, 
        # Mode can be one of {triangular, triangular2, exp_range}
        "mode" : "triangular", 
        "gamma" : 0.9995,
        "base_lr" : 0.0001, # 0.016, 
        "max_lr" :0.0005,  # 0.75
    },
    "training": {
        "stake_training_data": 0.75,
        "total_number" : 1000,
        "n_folds_cv": 6,
        "n_epochs" : 20,
        "patience" : 7,
    },
    "filed_location": {
        "model_for_test" : "../../../../models/cross_validation/MSE/phm67_",
        "log_file" : "../../../../models/cross_validation/MSE/phm67_log_", 
        "history_trainval" : "../../../visualisation/files/cross_validation/MSE/phm_trainval.csv",
        "history_best_configuration" : "../../../visualisation/files/cross_validation/MSE/phm67_configurations.csv"
    }
}

## Split Data into folds
- ignored features are getting removed
- remaining data are split up into folds

In [6]:
cv_provider = CrossValidationProvider(path=param["data"]["path"], 
                                      no_folds=param["training"]["n_folds_cv"], 
                                      amount_data=param["training"]["total_number"],
                                      ignored_features = param['preprocessing']['droped_features'],
                                      stake = param["training"]["stake_training_data"], 
                                     )
folds, test_set = cv_provider.provide_data()

## Cross Validation Training

In [7]:
# Initialise Logger
session_id = str(randint(10000, 99999))
logger = Logger(param["filed_location"]["log_file"], session_id)

# Create file where validation results are stored and add header
column_names_validation = ["validation_fold","validation_loss", "training_loss",
                           "n_hidden_lstm", "sequence_length", "n_lstm_layer", 
                           "n_hidden_fc"]
with open(param["filed_location"]["history_best_configuration"], "a+") as file:
    [file.write(column+";") for column in column_names_validation]
    file.write("\n")
         
for iteration in range (0, param["training"]["n_folds_cv"]):
    # Select folds for current iteration
    training_folds = [x for i,x in enumerate(folds) if i!=iteration] 
    validation_fold = folds[iteration]
    print("Validation Data : Fold "+ str(iteration+1))
    logger.log_message("Validation Data : Fold "+ str(iteration+1))
    print("- -"*20)
    logger.log_message("- -"*20)
    
    # Get mean and variance of training folds 
    raw_training_data = pd.concat(training_folds, axis = 0, ignore_index=True)
    raw_validation_data = validation_fold
    scaler = DataScaler(features_not_to_scale= param['preprocessing']['features_not_to_scale'])
    _, _ = scaler.scale_data(raw_training_data, raw_validation_data)
    mean_train, val_train = scaler.provide_statistics()
  
    # Scale all folds
    scaled_folds = []
    for i in range(len(folds)):
        scaled_folds.append(scaler.scale_fold(folds[i], mean_train, val_train))

    # Training model and test hyperparameter on validation set
    for n_lstm_layer in param["model"]["lstm_layer"]:
        for sequence_size  in param["model"]["sequence_size"]:
            # Creat Dataset of Training Folds
            scaled_folds_training = [x for i,x in enumerate(scaled_folds) if i!=iteration]
            reformated_folds = []
            for i in range(len(scaled_folds_training)):
                reformated_folds.append(DataSet(scaled_folds_training[i], timesteps=sequence_size))
            
            dataset_validation = DataSet(scaled_folds[iteration], timesteps=sequence_size)
            dataset_training = ConcatDataset(reformated_folds)
            
            # Initialize DataLoader
            data_loader_training = DataLoader(dataset_training, 
                                              batch_size=param["model"]["batch_size"], 
                                              num_workers=0, 
                                              shuffle=True, 
                                              drop_last=True
                                             )
            data_loader_validation = DataLoader(dataset_validation, 
                                                batch_size=param["model"]["batch_size"], 
                                                num_workers=0, 
                                                shuffle=True, 
                                                drop_last=True
                                               )
            
            for n_hidden_lstm in param["model"]["n_hidden_lstm"]:
                for n_hidden_fc in param["model"]["n_hidden_fc"]:
                    print("Start with new hyperparameters in grid search: ")
                    logger.log_message("Start with new hyperparameters in grid search: ")
                    print("Sequence_size: {}".format(sequence_size))
                    logger.log_message("Sequence_size: {}".format(sequence_size))
                    print("Number LSTM Layers: {}".format(n_lstm_layer))
                    logger.log_message("Number LSTM Layers: {}".format(n_lstm_layer))
                    print("LSTM Number Hidden Dimensions: {}".format(n_hidden_lstm))
                    logger.log_message("LSTM Number Hidden Dimensions: {}".format(n_hidden_lstm))
                    print("FC NN Number Hidden Dimensions: {}".format(n_hidden_fc))
                    logger.log_message("FC NN Number Hidden Dimensions: {}".format(n_hidden_fc))
                    
                    torch.manual_seed(0)
                    model = LstmMse(batch_size=param['model']['batch_size'], 
                                    input_dim=param['model']['input_size'], 
                                    n_hidden_lstm=n_hidden_lstm, 
                                    n_layers=n_lstm_layer,
                                    dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                                    dropout_rate_fc= param['model']['dropout_rate_fc'],
                                    n_hidden_fc=n_hidden_fc
                                    )

                    # Define Loss Function
                    criterion = LossMse(param["model"]["input_size"], param["model"]["batch_size"])

                    # Initialize Optimizer and Cyclic Learning Rate Scheduler
                    optimizer = torch.optim.SGD(model.parameters(), lr=1.)  
                    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer=optimizer, 
                                                                  base_lr=param['cycling_lr']['base_lr'], 
                                                                  max_lr=param['cycling_lr']['max_lr'], 
                                                                  step_size_up=(raw_training_data.shape[0]/8)*2, 
                                                                  mode=param['cycling_lr']['mode'],
                                                                  gamma=param['cycling_lr']['gamma']
                                                                  )
                    # Initialize Trainer
                    trainer = Trainer(model=model,
                                      optimizer=optimizer,
                                      scheduler=scheduler,
                                      scheduler_active = param["cycling_lr"]["scheduler_active"],
                                      criterion=criterion, 
                                      location_model=param["filed_location"]["model_for_test"], 
                                      patience=param['training']['patience']
                                     )

                    # Measure training time for current configuration
                    start = time.time()
                    for epoch in range(param['training']['n_epochs']):
                        # Train
                        mean_epoch_training_loss = trainer.train(data_loader_training)

                        # Evaluate
                        mean_epoch_validation_loss = trainer.evaluate(data_loader_validation, epoch)
                        
                        if mean_epoch_validation_loss < trainer.lowest_val_loss:
                            trainer.trials = 0
                            trainer.lowest_val_loss = mean_epoch_validation_loss
                            trainer.lowest_train_loss = mean_epoch_training_loss
                            status_ok = True
                            
                        else:
                            trainer.trials += 1
                            if trainer.trials >= trainer.patience:
                                status_ok = False       

                        if not status_ok or epoch == (param['training']['n_epochs'])-1:
                            logger.log_message("Stopped training on this partion at epoch {}".format(epoch))
                            print("Stopped training on this partion at epoch {}".format(epoch))
                            logger.log_message("Lowest training loss for this configuration: {}".format(trainer.lowest_train_loss))
                            print("Lowest training loss for this configuration: {}".format(trainer.lowest_train_loss))
                            logger.log_message("Lowest validation loss for this configuration: {}".format(trainer.lowest_val_loss))
                            print("Lowest validation loss for this configuration: {}".format(trainer.lowest_val_loss))
                            
                            # Statistics of current fold
                            statistics_validation = ["fold "+str(iteration+1),
                                                     trainer.lowest_val_loss,
                                                     trainer.lowest_train_loss,
                                                     n_hidden_lstm, 
                                                     sequence_size,
                                                     n_lstm_layer, 
                                                     n_hidden_fc
                                                     ]

                            # Safe statistics to .csv file
                            with open(param["filed_location"]["history_best_configuration"], "a") as file:
                                for value in statistics_validation:
                                    file.write(str(value)+";")
                                file.write("\n")
                            break       
                    print("\n"+"# #"*20+"\n")
                    logger.log_message("\n"+"# #"*20+"\n")
print("Finished Cross-Validation")
logger.log_message("Finished Cross-Validation")

Validation Data : Fold 1
- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -
Start with new hyperparameters in grid search: 
Sequence_size: 10
Number LSTM Layers: 1
LSTM Number Hidden Dimensions: 10
FC NN Number Hidden Dimensions: 60
Stopped training on this partion at epoch 3
Lowest training loss for this configuration: 0.9318259647116065
Lowest validation loss for this configuration: 1.3952604443899224

# ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## #

Start with new hyperparameters in grid search: 
Sequence_size: 10
Number LSTM Layers: 1
LSTM Number Hidden Dimensions: 50
FC NN Number Hidden Dimensions: 60
Stopped training on this partion at epoch 3
Lowest training loss for this configuration: 0.9414016139560513
Lowest validation loss for this configuration: 1.4271113180156265

# ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## #

Start with new hyperparameters in grid search: 
Sequence_size: 50
Number LSTM Layers: 1
LSTM Number Hidden Dimensions: 10
FC N

## Train Model with best hyperparameter

In [20]:
# Best hyperparameter (look into result csv file and set values accordingly)
opt_n_hidden_lstm = 20
opt_sequence_size = 20
opt_n_lstm_layer = 1
opt_n_hidden_fc = 40
epochs_for_final_train = 2

In [23]:
# Training Data
raw_training_data = pd.concat(folds, axis = 0, ignore_index=True)

# Scale training data and test data (test data with mean and variance of training data)
scaler = DataScaler(features_not_to_scale= param['preprocessing']['features_not_to_scale'])
train_data_scaled, test_data_scaled = scaler.scale_data(raw_training_data, test_set)

# Initialize DataSet
dataset_train = DataSet(train_data_scaled, timesteps=opt_sequence_size)

# Initialize DataLoader
data_loader_training = DataLoader(dataset_train, 
                                  batch_size=param["model"]["batch_size"], 
                                  num_workers=0, 
                                  shuffle=True, 
                                  drop_last=True
                                 )

# Initialise Model with best hyperparameter
torch.manual_seed(0)
model = LstmMse(batch_size=param['model']['batch_size'], 
                input_dim=param['model']['input_size'], 
                n_hidden_lstm=opt_n_hidden_lstm, 
                n_layers=opt_n_lstm_layer,
                dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                dropout_rate_fc= param['model']['dropout_rate_fc'],
                n_hidden_fc=opt_n_hidden_fc,
                )

# Define Loss Function
criterion = LossMse(param["model"]["input_size"], param["model"]["batch_size"])

# Initialize Optimizer and Trainer
optimizer = torch.optim.SGD(model.parameters(), lr=1.)  
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer=optimizer, 
                                              base_lr=param['cycling_lr']['base_lr'], 
                                              max_lr=param['cycling_lr']['max_lr'], 
                                              step_size_up=(len(raw_training_data)/param['model']['batch_size'])*2, # Authors of Cyclic LR suggest setting step_size 2-8 x training iterations in epoch.
                                              mode=param['cycling_lr']['mode'],
                                              gamma=param['cycling_lr']['gamma']
                                             )
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  scheduler=scheduler,
                  scheduler_active = param["cycling_lr"]["scheduler_active"],
                  criterion=criterion, 
                  location_model=param["filed_location"]["model_for_test"], 
                  patience=param['training']['patience']
                 )

logger.log_message("\n")
print("Training phase of final model started")
logger.log_message("Training phase of final model started")

for epoch in range(epochs_for_final_train):
    # Train with batches 
    mean_epoch_training_loss = trainer.train(data_loader_training)

    # Save model if its the best one since the last change in configuration of hyperparameters
    status_ok = trainer.save_model(epoch, mean_epoch_training_loss, session_id)
    
    if not status_ok or epoch == epochs_for_final_train-1:
        print("Training phase is finished with training loss: {}".format(mean_epoch_training_loss))
        logger.log_message("Training phase is finished with training loss: {}".format(mean_epoch_training_loss))
        break

Training phase of final model started
Epoch 0: best model saved with loss: 0.9624240054083723
Epoch 1: best model saved with loss: 0.8821538669722421
Training phase is finished with training loss: 0.8821538669722421


## Test trained model

In [25]:
model = LstmMse(batch_size=param['model']['batch_size'], 
                input_dim=param['model']['input_size'], 
                n_hidden_lstm=opt_n_hidden_lstm, 
                n_layers=opt_n_lstm_layer,
                dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                dropout_rate_fc= param['model']['dropout_rate_fc'],
                n_hidden_fc=opt_n_hidden_fc,
                )

checkpoint = torch.load(param["filed_location"]["model_for_test"]+"id"+str(session_id))
model.load_state_dict(checkpoint['model_state_dict'])

# Initialize DataSet
dataset_test = DataSet(test_data_scaled, timesteps=opt_sequence_size)

# Initialize DataLoader
data_loader_test = DataLoader(dataset_test, 
                              batch_size=param["model"]["batch_size"], 
                              num_workers=0, 
                              shuffle=True, 
                              drop_last=True
                             )

# Define Loss Function
criterion = LossMse(param["model"]["input_size"], param["model"]["batch_size"])

# Initialize Tester
tester = Tester(model=model, criterion=criterion)

# Evaluate Testset
mean_test_loss = tester.evaluate(data_loader_test)
print("\n"+"# #"*20+"\n")
logger.log_message("\n"+"# #"*20+"\n")
print("Mean loss of test dataset is {}".format(mean_test_loss))
logger.log_message("Mean loss of test dataset is {}".format(mean_test_loss))
print("\n"+"# #"*20+"\n")
logger.log_message("\n"+"# #"*20+"\n")
print("Cross Validation finished")
logger.log_message("Cross Validation finished")
# Delet model
os.remove(param["filed_location"]["model_for_test"]+"id"+str(session_id))


# ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## #

Mean loss of test dataset is 0.6929679290668385

# ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## #

Cross Validation finished
